In [1]:

import os
import pickle, sys
import torch, torchvision
from img_text_composition_models import TIRG
from datasets import Fashion200k

In [4]:
embed_dim = 512
dataset_path = '/Users/bo/Downloads/200k/'

In [5]:

trainset = Fashion200k(path=dataset_path, 
                       split='train',        
                        transform=torchvision.transforms.Compose([
                        torchvision.transforms.Resize(224),
                        torchvision.transforms.CenterCrop(224),
                        torchvision.transforms.ToTensor(),
                        torchvision.transforms.Normalize([0.485, 0.456, 0.406],
                                                         [0.229, 0.224, 0.225])
                    ]))

read pants_train_detect_all.txt
read dress_train_detect_all.txt
read jacket_train_detect_all.txt
read skirt_train_detect_all.txt
read top_train_detect_all.txt
Fashion200k: 172049 images
53099 unique cations
Modifiable images 106464


In [6]:
texts = [t for t in trainset.get_all_texts()]

In [7]:
model = TIRG(texts, embed_dim)
model_sd = torch.load('checkpoint.pth', map_location=torch.device('cpu'))
model.load_state_dict(model_sd['model_state_dict'])
_ = model.eval()

In [10]:
from PIL import Image
cur_dir = '/Users/bo/Downloads/work/opensource/examples/multimodal-search-tirg/TirgImageEncoder/tests/'
imgs = []

def transformer():
    return torchvision.transforms.Compose([
        torchvision.transforms.Resize(224),
        torchvision.transforms.CenterCrop(224),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406],
                                         [0.229, 0.224, 0.225])])

t = transformer()
for img_name in range(4):
    img_path = os.path.join(cur_dir, f'imgs/{img_name}.jpeg')
    with open(img_path, 'rb') as f:
        img = Image.open(img_path)
        img = img.convert('RGB')
        img = t(img)
        imgs.append(img)

In [11]:
imgs = torch.stack(imgs).float()

In [12]:
img_feature = model.extract_img_feature(imgs)

In [13]:
img_feature

tensor([[-0.0622, -2.4222, -4.3844,  ..., -1.0376,  3.1721, -2.0411],
        [-0.1847, -3.2721, -4.3416,  ...,  2.2188,  2.9847, -3.0952],
        [ 2.0852, -0.4260, -1.4903,  ..., -0.2095, -0.8310, -5.4079],
        [ 0.2074, -0.7349, -3.8016,  ...,  0.2713, -0.0056, -1.9443]],
       grad_fn=<AddmmBackward>)

In [15]:
import numpy as np

np.save('expected.npy', img_feature.detach().numpy())

In [17]:
for item in trainset:
    print(item.keys())
    break

dict_keys(['source_img_id', 'source_img_data', 'source_caption', 'target_img_id', 'target_img_data', 'target_caption', 'mod'])
